# Failed Banks ML Project

## Econ 1680: MLTA and Econ

#### Name: Lena Kim

Bank failure is often a harbinger of economic crises, as people look towards the status of
banks as a symbol of financial health. Classic economic literature has countlessly used banks’
financial statements and ratios to develop regression models that can predict a bank’s even-
tual closure (Meyer and Pifer (1970)). Such research has indicated liquidity ratios, rates of
interest, and operating revenue as the most salient metrics in determining whether a bank
fails.

For this project, I seek to go beyond the prediction of bank failures and consider what
happens after that failure. Using the failed bank’s balance sheet metrics, I hope to predict
whether it is eventually acquired by a larger national bank, a regional bank, or not acquired
at all. I will consider grouping by time, as the 2008 financial crisis will have significantly
played a hand in these results. In doing so, I will identify which balance sheet metrics play
the most significant role in this result, and compare these outcomes with the classic findings
and metrics defined previously.

Research Question: Given a failed bank’s balance sheet metrics, how can we predict
whether it is acquired by a national bank, a regional bank, or not acquired at all? Which
metrics contribute the most weight to this decision?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from math import sqrt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import normalize
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 
warnings.simplefilter(action='ignore', category= RuntimeWarning) 

In [3]:
balance_sheets = pd.read_csv('data/BalanceSheetSummaries.xls.csv')
acquisitions = pd.read_csv('data/banklist.csv', encoding='latin-1')

In [4]:
acquisitions.head()

,Bank Name,City,State,Cert,Acquiring Institution,Closing Date,Fund
0,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.",1-May-23,10543
1,Signature Bank,New York,NY,57053,"Flagstar Bank, N.A.",12-Mar-23,10540
2,Silicon Valley Bank,Santa Clara,CA,24735,FirstCitizens Bank & Trust Company,10-Mar-23,10539
3,Almena State Bank,Almena,KS,15426,Equity Bank,23-Oct-20,10538
4,First City Bank of Florida,Fort Walton Beach,FL,16748,"United Fidelity Bank, fsb",16-Oct-20,10537


In [5]:
#Data cleanup:
#Clean "failure date" to convert all types into datetime
balance_sheets['Failure Date'] = pd.to_datetime(balance_sheets['Failure Date'], format= '%m/%d/%Y', errors = 'coerce').combine_first(
    pd.to_datetime(balance_sheets['Failure Date'], format='%m-%d-%Y', errors='coerce')
) 

#Clean "Receivership column" into lowercase 
balance_sheets['Bank Name'] = balance_sheets['Receivership'].str.lower()
balance_sheets['Receivership'] = balance_sheets['Bank Name']
balance_sheets.head()

#Strip whitespaces of columns in acquisitions:
acquisitions.columns = acquisitions.columns.str.strip()
acquisitions['Bank Name'] = acquisitions['Bank Name'].str.lower()

#do a join to combine the two based on 'Receivership' column of balance_sheets and 'Bank Name' of acquisitions:
banks = balance_sheets.merge(acquisitions, left_on='Receivership', right_on='Bank Name', how='left')
#Get rid of repeated info: 
banks = banks.drop(columns=['Bank Name_x', 'Bank Name_y', 'Year']) #Year means "as of 2023", which is why all values are 2023. drop this. 

#Call head to ensure cleanliness:
banks.head(10)


,Fund Code,Receivership,Quarter,Failure Date,Cash and Investments,Due from FDIC Corp and Receivables,Assets in Liquidation,Estimated Loss on Assets in Liquidation(1),Total Assets,Administrative Liabilities,...,Dividends Paid to Date - Other Claimants,Dividends Paid to Date - Other Claimants %,Total Unpaid Other Claimants,Total Unpaid Other Claimants %,City,State,Cert,Acquiring Institution,Closing Date,Fund
0,10538,almena state bank,Q1,2020-10-23,2785.0,0.0,0.0,0.0,2785.0,0.0,...,0.0,0.0,338.0,100.0,Almena,KS,15426.0,Equity Bank,23-Oct-20,10538.0
1,10218,"amcore bank, na",Q1,2010-04-23,13944.0,0.0,0.0,0.0,13944.0,5350.0,...,0.0,0.0,57986.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
2,10057,america west bank,Q1,2009-05-01,1006.0,0.0,0.0,0.0,1006.0,1.0,...,0.0,0.0,1097.0,100.0,Layton,UT,35461.0,Cache Valley Bank,1-May-09,10057.0
3,10053,american southern bank,Q1,2009-04-24,2456.0,0.0,0.0,0.0,2456.0,0.0,...,0.0,0.0,232.0,100.0,Kennesaw,GA,57943.0,Bank of North Georgia,24-Apr-09,10053.0
4,10155,amtrust bank,Q1,2009-12-04,14112.0,0.0,0.0,0.0,14112.0,5352.0,...,0.0,0.0,25727.0,100.0,Cleveland,OH,29776.0,New York Community Bank,4-Dec-09,10155.0
5,10369,atlantic bank & trust,Q1,2011-06-03,1018.0,0.0,0.0,0.0,1018.0,0.0,...,0.0,0.0,33.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
6,10380,bank of choice,Q1,2011-07-22,1144.0,0.0,0.0,0.0,1144.0,1.0,...,0.0,0.0,860.0,100.0,Greeley,CO,2994.0,"Bank Midwest, N.A.",22-Jul-11,10380.0
7,10029,bank of clark county,Q1,2009-01-16,2520.0,0.0,0.0,0.0,2520.0,72.0,...,0.0,0.0,779.0,100.0,Vancouver,WA,34959.0,Umpqua Bank,16-Jan-09,10029.0
8,10064,bank of lincolnwood,Q1,2009-06-05,3013.0,0.0,0.0,0.0,3013.0,0.0,...,0.0,0.0,513.0,100.0,Lincolnwood,IL,17309.0,Republic Bank of Chicago,5-Jun-09,10064.0
9,10396,bank of the commonwealth,Q1,2011-09-23,5166.0,0.0,0.0,0.0,5166.0,9.0,...,0.0,0.0,1673.0,100.0,Norfolk,VA,20408.0,Southern Bank and Trust Company,23-Sep-11,10396.0


# Prep for ML

Here, I will add another column to my dataframe using "0" for not acquired/NA/nan, "1" for acquired by regional bank, and "2" for national bank

In [7]:
regional_banks = ['Cache Valley Bank', 'Bank of North Georgia', 'Columbia State Bank', 'North Jersey Community Bank', 'Morton Community Bank', 
'Century Bank of Georgia', 'CB&S Bank, Inc','Blackhawk Bank & Trust', 'Polonia Bank', 'Farmers and Merchants Bank', 'First Michigan Bank', 
'Northbrook Bank & Trust Company', 'The Savannah Bank, N.A.', 'The Jefferson Bank', 'Mutual of Omaha Bank', 'Whitney Bank', 'Metro City Bank',
'SmartBank', 'City National Bank', 'Stonegate Bank', 'Level One Bank', 'Ameris Bank', 'International Bank of Chicago', 'Providence Bank', 
'Nevada State Bank', 'First Federal Bank of Florida', 'Heritage Bank', 'MVB Bank, Inc.', 'Hamilton State Bank', 
'Bank 7', 'Kitsap Bank', 'Wheaton Bank & Trust'] #Manually classified as regional after research

national_banks = ['Equity Bank', 'New York Community Bank', 'Bank Midwest, N.A. ', 'Umpqua Bank', 'Republic Bank of Chicago', 'Southern Bank and Trust Company',
'U.S. Bank N.A.', 'MidFirst Bank', 'Banco Popular de Puerto Rico', 'Sadabell United Bank, N.A.', 'Manufacturers and Traders Trust Company (M&T Bank)',
'CenterState Bank of Florida, N.A.', 'First-Citizens Bank & Trust Company', 'United Fidelity Bank, fsb', 'First Merchants Bank, National Association', 
'Citizens Bank', 'Synovus Bank', 'Superior Bank, National Association', 'Union Bank, N.A.', 'Bank of the Ozarks', 'Valley National Bank', 
'East West Bank', 'Huntington National Bank']
#Manually classified as national after research

#Make new column, Acquisition Type, which will be our main target variable of interest:
banks['Acquisition Type'] = banks['Acquiring Institution'].apply(lambda x: 
                              0 if pd.isna(x) else 
                              1 if x in regional_banks else 
                              2 if x in national_banks else 
                              0)


In [8]:
#Train Test Split and Scaling Features:
banks['Total Liabilities'] = banks['Total Assets'] - banks['Net Worth (Deficit)'] #make total liabilities column from info already there

selected_columns = ['Cash and Investments', 'Due from FDIC Corp and Receivables', 'Assets in Liquidation', 'Total Assets', 
                    'Administrative Liabilities', 'Total Unpaid Other Claimants', 'Uninsured Deposit Claims',
                    'General Creditor', 'Total Liabilities',
                    'Acquisition Type'] #These are most important columns that are not basically repeated info
banks_relevant = banks[selected_columns]

X = banks_relevant.drop('Acquisition Type', axis=1) #features: everything except target
y = banks_relevant['Acquisition Type'] #target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 1680)

